# Homework - Neural networks - Part E (30 points)
## Artificial grammar learning with a simple recurrent network 

by *Brenden Lake* and *Todd Gureckis*  
Computational Cognitive Modeling  
NYU class webpage: https://brendenlake.github.io/CCM-site/  
email to course instructors: instructors-ccm-spring2019@nyuccl.org

<div class="alert alert-danger" role="alert">
This homework is due before midnight on Monday, Feb. 25, 2019.
</div>

In this notebook, you will train a recurrent neural network in PyTorch to learn an artificial grammar known as the "Reber grammar" (Reber, 1976). The Reber grammar is a classic learning task in psychology, and it is also a great domain for getting your hands dirty with recurrent neural networks.

The goal of this assignment is to build intuition about the training and operation of recurrent neural networks. We will use a Simple Recurrent Network (SRN) as proposed in Elman (1990). Conceptually, SRNs are similar to the very popular LSTM and GRU networks in contemporary machine learning. We certainly could use more these complex types of recurrent networks instead, but we don't need the additional power here.

<div class="alert alert-info">
This assignment is adapated from the online <a href="https://web.stanford.edu/group/pdplab/pdphandbook/handbookch8.html#x24-1060007">Parallel Distributed Processing (PDP) Handbook</a> by James McClelland, which has excellent background material on SRNs and their application to the Reber grammar task. We strongly suggest that you read this background information now (Section 7.1 of the PDP Handbook).
</div>

The original research was reported in Servan-Schreiber, Cleeremans, and McClelland (1991) as an investigation into the properties of recurrent neural networks. The SRN was also used been studied as a model of implicit sequence learning in humans, which was also investigated with the Reber grammar (Cleeremans and McClelland, 19991). We refer you to these references for more details if you are interested, especially in how RNNs can be used as a cognitive model of sequence learning.

**References:**
* Cleeremans, A. and McClelland, J. L. (1991). Learning the structure of event sequences. J Exp Psychol Gen, 120:235–253. 
* Elman, J. L. (1990). Finding structure in time. Cognitive Science, 14:179–211. 
* Servan-Schreiber, D., Cleeremans, A., and McClelland, J. L. (1991). Graded state machines: The representation of temporal contingencies in simple recurrent networks. Machine Learning, 7:161–193.

**Additional resources:**  
<div class="alert alert-info">
Some of this code was adapted from a <a href="http://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html">PyTorch recurrent network tutorial</a>. Please use this as an additional resource.
</div>

## Reber grammar

<img src="images/reber.png" style="width: 500px;"/>

This is the Reber grammar that we will be attempting to master with the SRN. The Reber grammar defines a set of allowable or "grammatical" sequences. The SRN will learn a sequence prediction task: what is the next element/symbol in a sequence, given the past elements. If the SRN predictions are aligned with the grammar, we can say that the SRN has implicitly learned to behave like the grammar, although without explicitly learning rules!

The Reber grammar is diagrammed above as a finite state machine (FSM) with six nodes (states) indicated by \#0,\#1,...,\#5. The FSM creates a string by traversing a path through the graph and visiting various nodes. It can transition between nodes that are connected by a directed edge, and when traversing an edge the FSM emits the output symbol associated with that edge. In the Reber grammar, each string begins with 'B' and ends with 'E'.

Let's go over an example. Starting at 'Begin', let's trace a path through the following nodes, \#0, \#1, \#3, and \#5, now ending at 'End.' This would create the string 'BTXSE' (please confirm for yourself). Other paths are possible, and self-connections and loops allow the FSM to create an infinite set of strings. For example, we can retrace the same path as before, but following the self-connection at node \#1, to create the path \#0, \#1, \#1, \#3, and \#5. This creates the string 'BTSXSE.' Likewise, we could keep adding loops, creating 'BTSSXSE', 'BTSSSXSE', ...., 'BTSSSSSSSXSE' and so on. All of these strings are considered "grammatical" according to the Reber grammar. A string that cannot be produced by the grammar is called "ungrammatical."

Here is are some initial exercises to check your understanding of the Reber grammar.

<div class="alert alert-success" role="alert">
<h3> Problem 1 (5 points) </h3>
<br>
Which of the following strings are grammatical? For each string below, please write 'YES' (for grammatical) or 'NO' (for ungrammatical)
<ul>
<li>BTSSXSSE</li>
<li>BTXXVPSE</li>
<li>BTXXVPXVPSE</li>
<li>BTXXXVPXVPSE</li>
<li>BTXXTVPXVPSE</li>
</ul>
<br>
Write your answers in a new Markdown cell below this one.
</div>

WRITE YOUR RESPONSE HERE

<div class="alert alert-success" role="alert">
<h3>Problem 2 (5 points)</h3>

The Reber grammar was carefully designed to display some interesting qualities to make it a difficult learning task. In general, the best a learner can do is predict one of two possible successors for a given node (except at the end of the sequence). For instance, if the grammar is at state \#1, both 'S' and 'X' are valid next symbols.
<br><br>
Note that, except for the special beginning and end symbols 'B' and 'E', *each symbol appears in two different places (on different edges) in the grammar.* Therefore, for a learner aiming to master the grammar and make the best  possible predictions, she cannot *just* pay attention to the previous symbol. This strategy does not uniquely identify the right set of possible next symbols. Instead, she must track the history further back, in order to make optimal predictions.
<br><br>
In this problem, your job is to simulate a "first-order" memory predictor, meaning you can only remember one symbol back (we also call this the "first-order statistics" of the domain). For each of these symbols, {'B', 'T', 'S', 'X', 'P', 'V'}, what is the set of possible successors given just a first-order memory?
<br><br>
*Hint: 'B' has two possible successors. Three of the other symbols have the same exact set of successors.*
<br><br>
Write your answers in a new Markdown cell below this one.
</div>

WRITE YOUR REPONSE HERE

## Loading the data

Now let's dive into some code. In our simulations, we are going to limit consideration to all the strings that are grammatical, with a limit on length of 10 symbols or less. There are 43 such strings, divided into a 21 strings [training set](data/reber_train.txt) and a 22 strings [test set](data/reber_test.txt) (Servan-Schreiber et al., 1991). Follow the links to see the training and test sets.

The following code first loads the training and test sets. We also define a function `seqToTensor` that takes a sequence of symbols and converts them to a 3D PyTorch tensor of size (seq_length x 1 x n_letters). The first dimension iterates over each symbol in the sequence, and each symbol is encoded as a one-hot vector (a 1 x n_letters tensor) indicating which symbol is present.

Execute the code below to see an example tensor for the sequence `BTSXSE`.

In [ ]:
# Let's start with some packages we need
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

# Get the index of 'letter' in 'all_letters'
def letterToIndex(letter):
    return all_letters.find(letter)

# Turn a sequence of symbols into a tensor <seq_length x 1 x n_letters>,
# where each symbol is a one-shot vector
def seqToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

# load the training and test data
with open('data/reber_train.txt','r') as fid:
    lines = fid.readlines()
    strings_training = [l.strip() for l in lines]

with open('data/reber_test.txt','r') as fid:
    lines = fid.readlines()
    strings_test = [l.strip() for l in lines]

#
strings_all = strings_training + strings_test
all_letters = ''.join(set(''.join(strings_training))) # string containing all possible characters
n_letters = len(all_letters) # total number of possible symbols
training_pats = [seqToTensor(s) for s in strings_training]
ntrain = len(training_pats)
print('all possible letters: %s' % all_letters)
print('first training pattern: %s' % strings_training[0])
print('tensor representation of %s:' % strings_training[0])
print(training_pats[0])

<img src="images/srn.png" style="width: 400px;"/>

## SRN architecture 
This is the SRN architecture and its implementation in PyTorch. The goal of the SRN is to predict the next symbol in the sequence, based on previous sequence of symbols it is provided. For instance, it may read 'B', then it may read 'T', and then try to predict the next element which could be either 'S' or 'X', as defined by the Reber grammar.

The SRN has an input, hidden, and output layer. Each unit in the input and output layer corresponds to a different symbols (see Figure). We use a softmax output layer to ensure that the network's prediction is a valid probability distribution over the set of possible symbols which could occur next.

The `SRN` class processes a sequence (and makes predictions) one symbol at a time. Since the hidden layer is recurrent, the hidden activations from the previous symbol are passed to the `forward` method as the tensor `hidden`, along with the next symbol to be processed as the tensor `input` (The previous hidden state is shown as the "Context units" in the above diagram). The output is the predicted probability of the next symbol (in the code below, `output` is the log-probability vector).

In [ ]:
class SRN(nn.Module):
    
    def __init__(self, nsymbols, hidden_size):
        #  nsymbols: number of possible input/output symbols
        super(SRN, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(nsymbols + hidden_size, hidden_size)
        self.h2o = nn.Linear(hidden_size, nsymbols)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        # input: [1 x nsymbol tensor] with one-hot encoding of a single symbol
        # hidden: [1 x hidden_size tensor] which is the previous hidden state
        combined = torch.cat((input, hidden), 1) # tensor size 1 x (nsymbol + hidden_size)
        hidden = self.i2h(combined) # 1 x hidden size
        hidden = F.sigmoid(hidden)
        output = self.h2o(hidden) # 1 x nsymbol
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size, requires_grad=True)

## Function for training the network
This code provides a `train` function for processing a sequence during training. First, the hidden layer is initialized as zeros `hidden = rnn.initHidden()` and the gradient is cleared with `rnn.zero_grad()`.

In the loop `for i in range(seq_length-1):`, the SRN processes each element in the sequence. After the network makes a prediction at each step, the backpropagation algorithm is used to compute the gradients `loss.backward()` and then the `optimizer` (defined later) updates the weights through `optimizer.step()`. 

The final `loss` is the negative log-likelihood of the predicted symbol, averaged across each step of prediction.

In [ ]:
# Turn a one-hot Variable of <1 x n_letters> into a Tensor of size <1> 
def variableToIndex(tensor):
    idx = np.nonzero(tensor.numpy().flatten())[0]
    assert len(idx) == 1
    return torch.tensor(idx, dtype=torch.long)

def train(seq_tensor, rnn):
    # seq_tensor: [seq_length x 1 x nsymbols tensor]
    # rnn : instance of SRN class
    hidden = rnn.initHidden()
    rnn.train()
    rnn.zero_grad()
    loss = 0
    seq_length = seq_tensor.shape[0]
    for i in range(seq_length-1):
        output, hidden = rnn(seq_tensor[i], hidden)
        loss += criterion(output, variableToIndex(seq_tensor[i+1]))
    loss.backward()
    optimizer.step()
    return loss.item() / float(seq_length-1)

## Functions for evaluating the network
The `eval_viz` function can be used to visualize the sequences of internal states of the network as it processes a string. `eval_viz` is similar to the `train` function above, except that it takes a string `seq` as input and displays the internal state of the network at every step of processing. 

Importantly, unlike `train` there are no weight updates. There is no need to study how this function works, it's just to visualize the network state. We'll explain how to use it later.

In [ ]:
def eval_viz(seq, rnn):
    # seq: string of symbols
    if not all(c in all_letters for c in seq):
        raise Exception('Input sequence contains an invalid letter')
    seq_tensor = seqToTensor(seq)
    rnn.eval()
    hidden = rnn.initHidden()
    seq_length = seq_tensor.shape[0]
    for i in range(seq_length):
        output, hidden = rnn(seq_tensor[i], hidden)
        print('Current input symbol: %s' % seq[i])
        if seq[i] != 'E':
            mydisplay(seq_tensor[i], output, hidden)

def mydisplay(input,output,hidden):
    v_extract = lambda x : x.data.numpy().flatten()
    output_prob = np.exp(v_extract(output))
    print('  Input:', end=' ')
    display_io_v(v_extract(input))
    print('Predicted next symbol:')
    print('  Hidden:', end=' ')
    display_v(v_extract(hidden))
    print('  Output:', end=' ')
    display_io_v(output_prob)
    print(" ")

def display_io_v(v):
    assert len(v)==len(all_letters)
    for idx,c in enumerate(all_letters):
        print('%s:%2.2f' % (c, v[idx]), end=' ')
    print('')

def display_v(v):
    for vi in v:
        print('%2.2f' % (vi), end=' ')
    print('')


Similarly, the `eval_set` function takes a list of strings `list_seq` as input and computes the average accuracy of the network's predictions across the list of strings.

How is performance defined? At each step of a string, the SRN's output is considered "correct" only if the highest scoring output symbol is valid according to the grammar. The `eval_set` function returns the average accuracy across the list of strings and elements of each string. 

As above, the details of how this function works are not that important. It just defines a reasonable measure of accuracy.

In [ ]:
def eval_set(list_seq, rnn, strings_possible):
    # list_seq : list of strings to process
    # rnn : instance of SRN class
    # strings_possible : list of all possible strings 
    n = len(list_seq)
    acc = np.zeros(n)
    for i in range(n):
        acc[i] = eval_pat(list_seq[i],rnn, strings_possible)
    return np.mean(acc)

def eval_pat(seq, rnn, strings_possible):
    # seq: string of symbols to process
    if not all(c in all_letters for c in seq):
        raise Exception('Input sequence contains an invalid letter')
    seq_tensor = seqToTensor(seq)
    rnn.eval()
    hidden = rnn.initHidden()
    seq_length = seq_tensor.shape[0]
    correct = 0.0
    history = seq[0]
    for i in range(seq_length-1):
        output, hidden = rnn(seq_tensor[i], hidden)
        widx = output.detach().topk(1)[1].item() # index of most likely successor, according to network
        myhistory = history + all_letters[widx] # history with next letter attached       
        context = [s[:len(myhistory)] for s in strings_possible] # all possible sequences of this length
        if myhistory in context: # was this a valid continuation?
            correct += 1.
        history += seq[i+1]
    correct = 100. * correct / (seq_length-1)
    return correct

## Training the simple recurrent network
Finally, we are ready to train the network! 

We create a SRN with 8 hidden units and train it for 500 epochs (sweeps through the training data). 

*Please run the network now!*

After the 500 epochs, the accuracy on the training and test set should be pretty good (above 95% correct). These numbers are computed every 20 epochs using our `eval_set` function. The training loss should be around 0.60. 

These numbers show that the network has mastered the structure of the grammar, predicting a reasonable successor symbol at each step of the sequence. It, however, can never be a perfect predictor. It is impossible to exactly predict each successor symbol, since there are almost always two possibilities.

In [ ]:
nepochs = 500 # number of passes through the entire training set 
nhidden = 8 # number of hidden units
learning_rate = 0.01

rnn = SRN(n_letters,nhidden) # create the network
optimizer = torch.optim.SGD(rnn.parameters(), lr=learning_rate) # stochastic gradient descent
criterion = nn.NLLLoss() #log-likelihood loss function

for myiter in range(1,nepochs+1): # for each epoch
    permute = np.random.permutation(ntrain)
    loss = np.zeros(ntrain)
    for p in permute:
        pat = training_pats[p]
        loss[p] = train(pat, rnn)
    if myiter % 20 == 0 or myiter == 1 or myiter == nepochs:
        train_acc = eval_set(strings_training, rnn, strings_all)
        test_acc = eval_set(strings_test, rnn, strings_all)
        print("epoch  %s: train loss %2.2f; train accuracy %2.2f; test accuracy %2.2f" % (myiter, np.mean(loss), train_acc, test_acc))

## Visualizing how a sequence is processed
Now let's understand what the network has learned!

Let's process the test string `BPTVVE` with the `eval_viz` function (see code in next cell). The function will walk you through each step of the sequence, and show you the network's internal state and predicted output. For instance, for the first symbol 'B', you will see a display roughly like:

```
Current input symbol: B
  Input : B:1.00 E:0.00 P:0.00 S:0.00 T:0.00 V:0.00 X:0.00 
Predicted next symbol:
  Hidden: 0.21 0.13 0.06 0.65 0.93 0.60 0.15 0.95 
  Output: B:0.00 E:0.00 P:0.42 S:0.00 T:0.57 V:0.01 X:0.00 
```
  
The 'Input' pattern shows the one-hot encoding of 'B'.    
The 'Hidden' pattern shows the activation of the hidden layer.   
The 'Output' pattern shows the prediction, in terms of probabilities of each of the successor symbols.  

In this case, the network is splitting its bet between 'P' and 'T' when predicting the next symbol, which is exactly right. Note that the network will not always choose exactly 50/50 when predicting successors, since the training patterns are not perfectly balanced in this way.

<div class="alert alert-success" role="alert">

<h3>Problem 3 (5 points)</h3>
<br>
Execute the code below to consider each step of the test sequence `'BPTVVE'`. Does the network make good predictions at each step? Write two or three sentences about the performance and your reaction.

The model makes probabilistic predictions, so don't just analyze the "most probable" prediction at each step. Look at how the network is distributing its probability across multiple successors.

</div>

WRITE YOUR RESPONSE HERE

The model should be approximately correct at this point. It splits its probability between two valid successors at each step.

In [ ]:
eval_viz('BPTVVE',rnn)

## Analyzing the timecourse of learning

Cleeremans and McClelland (1991) ran a psychology experiment that studied how people implicitly learn the structure of the Reber grammar. They found that in the initial stages of learning, people's behavior was largely consistent with the first-order statistics, as computed by the first-order memory that you worked out in Problem 2. Thus, if the previous symbol was 'V', their behavior would be largely consistent with four possible successors (rather than just 2, if they had mastered the grammar), regardless of what had happened previously in the sequence.

In the problem below, you will examine the timecourse of learning in detail.

<div class="alert alert-success" role="alert">
<h3>Problem 4: (15 points)</h3>
<br>
Retrain the network for only **5 epochs** (set `nepochs = 5` a few cells up). Using the `eval_viz` function and by examining AT LEAST TWO different sequences from the **test set**, how would you describe what the network has learned? Please write your answer in a few sentences, using specific examples. Mention first-order statistics (or higher-order statistics) if appropriate.
<br><br>
    <strong>Hint:</strong> It is helpful to examine a sequence that has the same symbol appearing twice, but where the Reber grammar is visiting different nodes (e.g., 'P' on the way to visiting node \#2, and also 'P' on the way to visiting node \#3). Does the network make the same prediction in both places? Does looking at the pattern of hidden activations help?
<br><br>
Write your answers in a new Markdown cell below this one.
</div>

WRITE YOUR RESPONSE HERE

<div class="alert alert-success" role="alert">
Retrain the network again. This time train for **50 epochs**. Answer the same questions as above.
<br><br>
Write your answers in a new Markdown cell below this one.
</div>

WRITE YOUR RESPONSE HERE

<div class="alert alert-success" role="alert">
Retrain the network again, this time train for the full **500 epochs**. Answer the same questions as above.
<br><br>
You may want to go through this whole process twice, in case there is some variability in your network runs.
</div>

WRITE YOUR RESPONSE HERE

## Turning in homeworks

When you are finished with this notebook. Save your work in order to turn it in.  To do this select *File*->*Download As...*->*HTML*.

<img src="images/save-pdf.png" width="300">

You can turn in your assignments using NYU Classes webpage for the course (available on https://home.nyu.edu). **Make sure you complete all parts (A-E) of this homework.**